# Stock Market Data

In [1]:
# Import libraries and dependencies
import os
import requests
import json
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from pathlib import Path
import cbpro
from datetime import date

%matplotlib inline

### Load environment variables and import Alpaca keys

In [2]:
# Load the environment variables by calling the load_dotenv function
load_dotenv()

True

In [3]:
# Set Alpaca API key and secret by calling the os.getenv function and referencing the environment variable names
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Check the values were imported correctly by evaluating the type of each
display(type(alpaca_api_key))
display(type(alpaca_secret_key))

str

str

### Setup REST object

In [4]:
# Create an Alpaca API REST object by calling the tradeapi.REST function
# Set the parameters to alpaca_api_key, alpaca_secret_key and api_version="v2" 
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

### Use the Alpaca SDK to make an API call

In [5]:
# Create a list of tickers for traditional (non-crypto) securities
tickers = ["DIA", "NDAQ", "SPY"]

In [6]:
# Set the values for start_date and end_date using the pd.Timestamp function
# Set this all to the ISO format by calling the isoformat function 
start_date = pd.Timestamp('2020-10-01', tz = "America/New_York").isoformat()
end_date = pd.Timestamp('2021-10-01', tz = "America/New_York").isoformat()

In [7]:
# Set timeframe to one day (1D)
timeframe = "1D"

# Set number of rows to 1000 to retrieve the maximum amount of rows
limit_rows = 1000

In [8]:
# Use the Alpaca get_barset function to gather the price information for each ticker
stock_prices = alpaca.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date,
    limit = limit_rows
).df

# Review the first five rows of the resulting DataFrame 
display(stock_prices.head())
display(stock_prices.tail())

DIA                                     NDAQ  \
                             open    high     low   close   volume    open   
time                                                                         
2020-10-01 00:00:00-04:00  279.33  280.35  276.57  278.12  2599552  123.92   
2020-10-02 00:00:00-04:00  274.17  278.51  273.60  276.78  4217688  122.66   
2020-10-05 00:00:00-04:00  278.79  281.59  278.75  281.40  1678416  124.19   
2020-10-06 00:00:00-04:00  282.54  283.52  277.24  277.67  4233910  124.84   
2020-10-07 00:00:00-04:00  280.28  283.67  280.23  282.89  2212655  123.38   

                                                              SPY            \
                             high     low   close  volume    open      high   
time                                                                          
2020-10-01 00:00:00-04:00  124.57  122.87  123.92  908229  337.69  338.7400   
2020-10-02 00:00:00-04:00  124.67  122.26  123.53  528313  331.70  337.0126   
2020-10-05 00:00:00-04:00  124.99  122.70  124.88  539675  336.06  339.9600   
2020-10-06 00:00:00-04:00  125.79  123.09  123.57  517618  339.92  342.1700   
2020-10-07 00:00:00-04:00  124.80  122.18  122.97  666336  338.12  341.6300   

                                                     
                              low   close    volume  
time                                                 
2020-10-01 00:00:00-04:00  335.01  337.03  77057063  
2020-10-02 00:00:00-04:00  331.19  333.87  72797429  
2020-10-05 00:00:00-04:00  336.01  339.73  38540997  
2020-10-06 00:00:00-04:00  334.38  334.94  74231873  
2020-10-07 00:00:00-04:00  338.09  340.73  40579189

DIA                                     NDAQ  \
                             open    high     low   close   volume    open   
time                                                                         
2021-09-27 00:00:00-04:00  348.29  350.50  348.12  348.59  2962818  196.50   
2021-09-28 00:00:00-04:00  347.33  347.82  342.41  342.94  6500883  193.83   
2021-09-29 00:00:00-04:00  343.90  345.67  342.88  343.83  3839304  191.64   
2021-09-30 00:00:00-04:00  345.08  345.44  338.11  338.28  7371559  195.10   
2021-10-01 00:00:00-04:00  340.00  344.78  337.68  343.19  5920343  194.05   

                                                                 SPY  \
                              high     low    close   volume    open   
time                                                                   
2021-09-27 00:00:00-04:00  196.640  194.52  195.805   814123  442.81   
2021-09-28 00:00:00-04:00  194.330  189.38  190.960  1184653  439.69   
2021-09-29 00:00:00-04:00  194.410  190.92  194.000   772300  435.19   
2021-09-30 00:00:00-04:00  197.334  192.92  193.020   848464  436.02   
2021-10-01 00:00:00-04:00  194.750  190.66  193.840   670129  430.98   

                                                                
                               high     low   close     volume  
time                                                            
2021-09-27 00:00:00-04:00  444.0500  441.90  442.68   51611925  
2021-09-28 00:00:00-04:00  440.0400  432.94  433.74  114959576  
2021-09-29 00:00:00-04:00  437.0400  433.85  434.44   69828191  
2021-09-30 00:00:00-04:00  436.7700  428.78  429.17  115315276  
2021-10-01 00:00:00-04:00  436.0329  427.24  434.19  110774792